In [1]:
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
import os
import requests

load_dotenv()

True

In [2]:
summary_template = """
Given the information {information} about a person, I want you to create:
1. A short summary
2. Two interesting facts about them
"""

summary_prompt_template = PromptTemplate(
    input_variables=["information"],
    template=summary_template
)

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    temperature=0.8,
    streaming=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [4]:
chain = LLMChain(llm=llm, prompt=summary_prompt_template)
# res = chain.invoke(input={"information": information})

# print(res)

In [5]:

def scrape_linkedin_profile(linkedin_profile_url: str):
    """Scrape information from LinkedIn profiles
    Manually scrape the information from the LinkedIn profile"""
    api_endpoint = 'https://nubela.co/proxycurl/api/v2/linkedin'
    header_dic = {'Authorization': f'Bearer {os.environ.get("PROXYCURL_API_KEY")}'}

    response = requests.get(
        api_endpoint, params={"url": linkedin_profile_url}, headers=header_dic
    )

    # Clean null and unneeded data from response:

    data = response.json()
    data = {
        k: v
        for k, v in data.items()
        if v not in ([], "", "", None)
        and k not in ["people_also_viewed"] # can optionally add , "certifications" inside brackets (include comma and space after "people_also_viewed")
    }

    if data.get("groups"):
        for group_dict in data.get("groups"):
            group_dict.pop("profile_pic_url")

    return data

linkedin_data = scrape_linkedin_profile(linkedin_profile_url='https://www.linkedin.com/in/harrison-chase-961287118')
# print(linkedin_data)

In [6]:
print(chain.run(information=linkedin_data))

/Users/jasonclishe/JC/repos/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


**Summary:**

Harrison Chase is the Co-Founder and CEO of LangChain, a technology company focused on using machine learning for language understanding. He has a strong background in machine learning and software engineering, and has held previous roles at Robust Intelligence and Kensho Technologies.

**Interesting Facts:**

* Harrison Chase has a degree from Harvard University.
* Harrison Chase's interests include sports, machine learning, software engineering, and statistics.
